# QCDA

QCDA, short for Quantum Circuit Design Automation, is designed as a one-stop process to make a logical quantum circuit executable on some quantum device. Obstacles that we may face in this process could be divided into three parts - decomposing a routine method with basic gates, reducing the size and depth of the circuit, and mapping the logical qubits to the physical qubits with a given layout while adding fewer swap gates. Corresponding to these problems, we develop three sub-modules - synthesis, optimization, and mapping - for them.

In [ ]:
import random
import numpy as np
from scipy.stats import unitary_group
# Import everything about gates and circuits
from QuICT.core import *
from QuICT.core.gate import *

# QCDA-flow

Generally, a QCDA workflow defines as a list of methods that would execute sequentially. Some default sublists are preset in the class, which is helpful to average users but perhaps not always targeted to the input circuit. Experienced users could customize the sequence with other methods for their benefit.

In [ ]:
from QuICT.qcda.qcda import QCDA
from QuICT.qcda.synthesis.gate_transform import USTCSet

type_list = [GateType.rx, GateType.ry, GateType.rz, GateType.x, GateType.y, GateType.z, GateType.cx]
layout = Layout.load_file("../layout/ibmqx2_layout.json")

# Read circuit
circuit = Circuit(5)
circuit.random_append(typelist=type_list)
target = random.sample(range(5), 3)
CCRz(np.pi / 3) | circuit(target)
circuit.random_append(typelist=type_list)
target = random.sample(range(5), 3)
CSwap | circuit(target)
circuit.random_append(typelist=type_list)
matrix = unitary_group.rvs(2 ** 3)
target = random.sample(range(5), 3)
Unitary(matrix) | circuit(target)
circuit.random_append(typelist=type_list)
circuit.draw()

# Show circuit statistics between processes
qcda = QCDA()
qcda.add_default_synthesis(USTCSet)
qcda.add_default_optimization()
qcda.add_default_mapping(layout)
qcda.add_default_synthesis(USTCSet)
circuit_phy = qcda.compile(circuit)
circuit_phy.draw()

# Synthesis

In synthesis, we implement methods to decompose complex gates with basic gates and transform a circuit with a given instruction set.

## Clifford

Bravyi, S., Shaydulin, R., Hu, S., & Maslov, D.L. (2021). Clifford Circuit Optimization with Templates and Symbolic Pauli Gates. Quantum, 5, 580.

https://arxiv.org/abs/2105.02291

In [ ]:
from QuICT.qcda.synthesis import CliffordUnidirectionalSynthesizer

n = 10
prob_list = [1 / 9 for _ in range(6)] + [1 / 3]
circuit = Circuit(n)
circuit.random_append(10 * n, typelist=CLIFFORD_GATE_SET, probabilities=prob_list)
circuit.draw()
print(circuit)
CUS = CliffordUnidirectionalSynthesizer(strategy='greedy')
circuit_syn = CUS.execute(circuit)
circuit_syn.draw()
print(circuit_syn)

## Gate transform

In [ ]:
from QuICT.qcda.synthesis import GateTransform, USTCSet

n = 10
circuit = Circuit(n)
circuit.random_append(50, random_params=True)
circuit.draw()
print(circuit)
GT = GateTransform(USTCSet)
circuit_tran = GT.execute(circuit)
circuit_tran.draw()
print(circuit_tran)

## Multi-controlled Toffoli (mct)

Barenco, Bennett, Cleve, DiVincenzo, Margolus, Shor, Sleator, Smolin, & Weinfurter (1995). Elementary gates for quantum computation. Physical review. A, Atomic, molecular, and optical physics, 52 5, 3457-3467 .

https://arxiv.org/abs/quant-ph/9503016

He, Y., Luo, M., Zhang, E.J., Wang, H., & Wang, X. (2017). Decompositions of n-qubit Toffoli Gates with Linear Circuit Complexity. International Journal of Theoretical Physics, 56, 2350-2361.

https://link.springer.com/article/10.1007/s10773-017-3389-4

In [ ]:
# The last qubit is ancillary
from QuICT.qcda.synthesis import MCTOneAux

n = 5
circuit = Circuit(n)
MCT = MCTOneAux()
MCT.execute(n) | circuit
# Show CCCX as a single gate
circuit.draw()
print(circuit)

## Uniformly-controlled gate

Mottonen, M., & Vartiainen, J.J. (2006). Decompositions of general quantum gates. arXiv: Quantum Physics.

https://arxiv.org/abs/quant-ph/0504100

In [ ]:
from QuICT.qcda.synthesis import UniformlyRotation

n = 5
circuit = Circuit(n)
angles = [2 * np.pi * random.random() for _ in range(1 << (n - 1))]
URy = UniformlyRotation(GateType.ry)
URy.execute(angles) | circuit
circuit.draw()
print(circuit)

In [ ]:
from QuICT.qcda.synthesis import UniformlyUnitary

n = 5
circuit = Circuit(n)
unitaries = [unitary_group.rvs(2) for _ in range(1 << (n - 1))]
UUnitary = UniformlyUnitary()
UUnitary.execute(unitaries) | circuit
circuit.draw()
print(circuit)

## Unitary decomposition

Shende, V.V., Bullock, S.S., & Markov, I.L. (2006). Synthesis of quantum-logic circuits. IEEE Transactions on Computer-Aided Design of Integrated Circuits and Systems, 25, 1000-1010.

https://arxiv.org/abs/quant-ph/0406176

Drury, B., & Love, P.J. (2008). Constructive quantum Shannon decomposition from Cartan involutions. Journal of Physics A: Mathematical and Theoretical, 41, 395305.

https://arxiv.org/abs/0806.4015


In [ ]:
from QuICT.qcda.synthesis import UnitaryDecomposition

n = 5
circuit = Circuit(n)
mat = unitary_group.rvs(1 << n)
UD = UnitaryDecomposition(recursive_basis=2)
gates, _ = UD.execute(mat)
gates | circuit
# circuit.draw()
print(circuit)

# Optimization

In optimization, we implement methods to reduce the size and depth of circuits with different gates.

## Quipper's optimization

Nam, Y.S., Ross, N.J., Su, Y., Childs, A.M., & Maslov, D.L. (2017). Automated optimization of large quantum circuits with continuous parameters. npj Quantum Information, 4, 1-12.

https://arxiv.org/abs/1710.07345

In [ ]:
from QuICT.qcda.optimization import CliffordRzOptimization

type_list = [GateType.h, GateType.cx, GateType.rz]

n = 5
circuit = Circuit(n)
circuit.random_append(2000, typelist=type_list, random_params=True)
# circuit.draw()
print(circuit)
AO = CliffordRzOptimization(mode='light', verbose=False)
circuit_opt = AO.execute(circuit)
# circuit_opt.draw()
print(circuit_opt)

## Commutative optimization

In [ ]:
from QuICT.qcda.optimization import CommutativeOptimization

typelist = [GateType.rx, GateType.ry, GateType.rz, GateType.x, GateType.y, GateType.z, GateType.cx]
prob_list = [1 / 9 for _ in range(6)] + [1 / 3]

n = 5
circuit = Circuit(n)
circuit.random_append(rand_size=100, typelist=typelist, random_params=True, probabilities=prob_list)
circuit.draw()
print(circuit)
CO = CommutativeOptimization(deparameterization=True)
circuit_opt = CO.execute(circuit)
circuit_opt.draw()
print(circuit_opt)

## Symbolic Clifford optimization

Bravyi, S., Shaydulin, R., Hu, S., & Maslov, D.L. (2021). Clifford Circuit Optimization with Templates and Symbolic Pauli Gates. Quantum, 5, 580.

https://arxiv.org/abs/2105.02291

In [ ]:
from QuICT.qcda.optimization import SymbolicCliffordOptimization

prob_list = [1 / 9 for _ in range(6)] + [1 / 3]

n = 5
circuit = Circuit(n)
circuit.random_append(10 * n, typelist=CLIFFORD_GATE_SET, probabilities=prob_list)
circuit.draw()
print(circuit)
SCO = SymbolicCliffordOptimization()
circuit_opt = SCO.execute(circuit)
circuit_opt.draw()
print(circuit_opt)

## Template optimization

Iten, R., Sutter, D., & Woerner, S. (2019). Efficient template matching in quantum circuits. ArXiv, abs/1909.05270.

https://arxiv.org/abs/1909.05270

In [ ]:
from QuICT.lib.circuitlib import CircuitLib
from QuICT.qcda.optimization import TemplateOptimization

n = 5
template_list = CircuitLib.load_template_circuit()
circuit = Circuit(n)
typelist = [GateType.x, GateType.cx, GateType.ccx, GateType.h, GateType.s, GateType.t, GateType.sdg, GateType.tdg]
circuit.random_append(200, typelist=typelist)
circuit.draw()
print(circuit)
TO = TemplateOptimization(
    template_list=random.sample(template_list, 10),
    heuristics_qubits_param=[10],
    heuristics_backward_param=[3, 1]
)
circuit_opt = TO.execute(circuit)
circuit_opt.draw()
print(circuit_opt)

## CNOT optimization

We implement various methods to optimize CNOT circuits in different cases, with or without ancillary qubits and topology restrictions. Here we take methods with no ancilla and no topology as an example to demo CNOT optimization.

In [ ]:
from QuICT.qcda.optimization import CnotWithoutAncilla, CnotLocalForceBfs, CnotLocalForceDepthBfs

n = 10
circuit = Circuit(n)
circuit.random_append(20 * n, typelist=[GateType.cx])
# circuit.draw()
print(circuit)

CWA = CnotWithoutAncilla()
circuit_cwa = CWA.execute(circuit)
# circuit_cwa.draw()
print(circuit_cwa)

CLFB = CnotLocalForceBfs()
circuit_clfb = CLFB.execute(circuit)
# circuit_clfb.draw()
print(circuit_clfb)

CLFDB = CnotLocalForceDepthBfs()
circuit_clfdb = CLFDB.execute(circuit)
# circuit_clfdb.draw()
print(circuit_clfdb)

# Mapping

In mapping, we implement the algorithm based on the Monte-Carlo tree search.

In [ ]:
from QuICT.tools.interface import OPENQASMInterface
from QuICT.qcda.mapping import MCTSMapping

layout = Layout.load_file("./example/ibmq_casablanca.layout")
# circuit = OPENQASMInterface.load_file("./example/example_test.qasm").circuit
circuit = Circuit(7)
circuit.random_append(20, typelist=[GateType.cx])
circuit.draw()
print(circuit)
MCTS = MCTSMapping(layout=layout, init_mapping_method="naive")
circuit_map = MCTS.execute(circuit)
circuit_map.draw()
print(circuit_map)